# Server Test
> This is the code I am using to test the server. In this test, I will use 2 connections and attempt to make the join the same game.

In [51]:
import socket as s

## Connecting to server

I connect to the GO server on port 80(configured in docker).

In [52]:
conn = s.socket(s.AF_INET, s.SOCK_STREAM)
conn.connect(("127.0.0.1", 80))

## Sending packets

All packets must be suffixed with \n. This is a python implemtation of how to send them:

In [53]:
# Sends a packet, returns whatever socket.Send returns
def send_packet(content: bytes):
    global conn
    return conn.send(content + b'\n')


## Creating a game

I need to send a packet with 'c' as the first character to intialise a game:

In [54]:
send_packet(b'c')

2

In response, I should get a packet with the *game code*.

In [55]:
BUFFER_SIZE = 1024

def recieve_packet() -> bytes:
    return conn.recv(BUFFER_SIZE)

In [56]:
packet = recieve_packet()

In [57]:
trimmed = packet[1:]
import json
packetDict = json.loads(trimmed)

## User 2

I will attempt to join the game as another user

In [58]:
conn2 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn2.connect(("127.0.0.1", 80))

In [59]:
conn2.send(b'j' + b'{"Name":"' + packetDict['Name'].encode() + b'"}\n')
conn2.recv(BUFFER_SIZE)

b'Jsuccess\n'


## Playing the game
The two players should be able to share data with each other.

In [60]:
conn.send(
    b'f{"X":1, "Y":1, "Dx":999, "Dy":999}\n'
)

36

In [61]:
conn.recv(BUFFER_SIZE)

b'F{"Players":[{"Name":"suncabnjla","X":1,"Y":1,"Dx":999,"Dy":999},{"Name":"dxmixckato","X":0,"Y":0,"Dx":0,"Dy":0}],"State":1,"Host":null,"Name":"zkjazlhe"}\n'

subequent reads should have different values for player 1, becuase Dx and Dy move the player along like so

In [62]:
import time
time.sleep(1)

conn2.send(
    b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n'
)
conn2.recv(BUFFER_SIZE)

b'F{"Players":[{"Name":"suncabnjla","X":1,"Y":1,"Dx":999,"Dy":999},{"Name":"dxmixckato","X":1,"Y":1,"Dx":0,"Dy":0}],"State":1,"Host":null,"Name":"zkjazlhe"}\n'